### Image classification convolution neural network

<b> In this exercise we build an image classification model used to classify an image based on a class. A classic example is to determine whether an image is an apple or orange or in general of which fruit class the image belongs to.
We use tensorflow and keras to sequential model with convolution neural network with multi layers to achieve the desired goal.
</b>

<b> Lets first download the fruits dataset and then extract the data into test set and training set</b>

In [ ]:
!pip install kaggle

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "uname" # please use proper username 
os.environ['KAGGLE_KEY'] = "kaggle_key" # provide the kaggle key appropriately
download_to_folder = '/tmp/fruits'
!kaggle datasets download -d moltean/fruits -p download_to_folder

In [ ]:
## create the directories one each for training dataset and test datset

base_dir = '/tmp/fruits'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# train directory with fruits pictures
train_fruits_dir = os.path.join(train_dir, 'fruits')

# validation directory with fruits pictures
validation_fruits_dir = os.path.join(validation_dir, 'fruits')

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
## convolution neural network 

## we will feed images of specific sizes to our neural net for processing, we will use convolution + relu + maxpooling modules.

# Our input feature map is 150x150x3: 150x150 for the image pixels, and 3 for
# the three color channels: R, G, and B
img_input = layers.Input(shape=(150, 150, 3))
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
# Flatten feature map to a 1-dim tensor so we can add fully connected layers
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model = Model(img_input, output)

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [ ]:
## data preprocessing
## setting up data generators to read images into the source folder

# scale image by 1./255
training_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# train iamges in batches of 25
train_generator = training_datagen.flow_from_directory(
        train_dir,  # source directory
        target_size=(175, 175),
        batch_size=25,
        class_mode='categorical')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(175, 175),
        batch_size=25,
        class_mode='categorial')

In [ ]:
## train on all images

history = model.fit(
      train_generator,
      steps_per_epoch=100, 
      epochs=25,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

In [ ]:
### visualize the accuracy

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()